In [ ]:
import nest_asyncio
nest_asyncio.apply()

# import asyncio
import time
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, Markdown
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()  


In [2]:
import ipywidgets as widgets 
from IPython.display import display, Markdown

In [ ]:
sample_text = (
    "The new product launch exceeded expectations, with sales reaching 150% of the Q1 target. Key contributing factors included the revamped marketing strategy and strong positive customer reviews. However, supply chain bottlenecks led to minor delays for 10% of orders, an issue now being addressed by diversifying suppliers. The financial outlook for Q2 remains robust."
)

prompts = {
    "Summarizer_Simple": "Summarize this text: {text_input}",
    "Summarizer_Executive": (
        "You are an expert summarizer for busy executives. Summarize this for quick understanding, focusing on key decisions and outcomes: {text_input}"
    )
}


In [4]:
if "prompt_selector" not in globals():
    prompt_selector = widgets.Dropdown(options=["Summarizer_Simple", "Summarizer_Executive"], description='Prompt:')
    temp_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.1, description='Temp:')
    run_button = widgets.Button(description="Run Prompt")
    output_area = widgets.Output()

In [5]:
def run_selected_prompt_sync(b):
    output_area.clear_output(wait=True)
    selected_prompt_name = prompt_selector.value
    temperature = temp_slider.value
    template = prompts[selected_prompt_name]
    prompt = template.format(text_input=sample_text)

    # Use append_display_data inside output_area context
    with output_area:
        output_area.append_display_data(Markdown(f"**Running '{selected_prompt_name}' with temp={temperature}...**"))
        try:
            start = time.perf_counter()
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
            )
            latency = time.perf_counter() - start
            content = response.choices[0].message.content

            output_area.append_display_data(Markdown(f"**Response:**\n```\n{content}\n```"))
            output_area.append_display_data(Markdown(f"*Latency: {latency:.2f}s, Tokens: {response.usage.total_tokens}*"))

        except Exception as e:
            output_area.append_display_data(Markdown(f"**Error:** {e}"))


In [6]:
# Unbind previous click handlers to avoid duplicate executions
run_button._click_handlers.callbacks = []

# Re-bind the button to the handler
run_button.on_click(run_selected_prompt_sync)

# Display widgets
display(prompt_selector, temp_slider, run_button, output_area)


Dropdown(description='Prompt:', options=('Summarizer_Simple', 'Summarizer_Executive'), value='Summarizer_Simpl…

FloatSlider(value=0.5, description='Temp:', max=1.0)

Button(description='Run Prompt', style=ButtonStyle())

Output()